# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

Installs

In [1]:
!pip install -U datasets --quiet
!pip install ipywidgets --quiet
!pip install py7zr --quiet
!pip install transformers --quiet
!pip install torch --quiet
!pip install rouge-score --quiet
!pip install hf_xet --quiet

Importing dataset

In [2]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0")
# Take just a small subset for quick training
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(2000))
small_eval_dataset = dataset["validation"].shuffle(seed=42).select(range(200))

train_dataset = small_train_dataset

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

Tokenizer

In [4]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from torch.utils.data import DataLoader

tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-cnn_dailymail")
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-cnn_dailymail")

def tokenize_function(examples):
    return tokenizer(examples['article'], truncation=True, padding='max_length', max_length=1024)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

print(tokenized_datasets['train'][0])

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

Loading data

In [8]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

train_dataset = tokenized_datasets['train']
train_dataloader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    collate_fn=default_data_collator
)

Tuning

In [ ]:
import torch
from torch.optim import AdamW

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        # Move batch to device
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

Generate Summary

In [ ]:
# Inference
def generate_summary(article):
    inputs = tokenizer(article, return_tensors="pt", max_length=1024, truncation=True, padding=True)
    inputs = inputs.to(device)

    summary_ids = model.generate(inputs['input_ids'], max_length=200, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Test the summarizer
sample_article = dataset['test'][0]['article']
summary = generate_summary(sample_article)
print(f"Article: {sample_article[:500]}...")  # Print part of the article
print(f"Summary: {summary}")

Rouge score

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def evaluate_summary(reference, generated):
    scores = scorer.score(reference, generated)
    return scores

# Example evaluation
reference_summary = dataset['test'][0]['highlights']
generated_summary = generate_summary(sample_article)

scores = evaluate_summary(reference_summary, generated_summary)
print(scores)